In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
import copy



# 1. 파일 불러오기

In [ ]:
#df = pd.read_csv('bs.csv', encoding='cp949')
df = pd.read_csv('bs.csv', header=0, parse_dates=True, index_col='Date', encoding='cp949')

 pd.read_csv('파일 명') : 파일을 읽어오는 함수
 
 header=0 : 파일의 첫 번째 행을 열 이름으로 사용 
 
 parse_dates=True : 'Date' 열을 날짜 형식으로 파싱 
 
 index_col='Date' : 'Date' 열을 인덱스로 설정
 
  encoding='cp949' : 인코딩 시 한국어를 인코딩하는 방식


In [36]:
df.head()

,수입(TEU),수출(TEU),수입환적(TEU),수출환적(TEU),수입물가지수,usd비율(Yuan/US),Dubai국제유가(US/배럴),강수량(mm),평균기온,평균최저기온,...,Unnamed: 116,Unnamed: 117,Unnamed: 118,Unnamed: 119,Unnamed: 120,Unnamed: 121,Unnamed: 122,Unnamed: 123,Unnamed: 124,Unnamed: 125
Date,,,,,,,,,,,,,,,,,,,,,
2014-01-01,"353,915.75","365,515.00","408,239.00","389,979.75",128.15,6.105,104.0,9.6,5.0,1.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-02-01,"334,478.00","334,191.25","313,752.00","350,400.25",128.34,6.121,105.0,44.1,5.8,2.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-03-01,"405,896.25","402,860.75","410,435.50","383,563.50",127.66,6.152,104.4,136.7,9.8,6.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-04-01,"391,681.50","402,606.75","390,004.00","401,684.25",127.56,6.158,104.6,147.8,14.6,11.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-05-01,"396,793.75","395,188.75","402,278.75","388,978.25",127.70,6.169,105.6,145.9,18.7,15.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# 2. 결측치 제거

In [37]:
print('전체 데이터 수 :', len(df))

전체 데이터 수 : 115


In [38]:
print('컬럼 별 결측치 개수')
len(df) - df.count()

컬럼 별 결측치 개수


수입(TEU)           1
수출(TEU)           1
수입환적(TEU)         1
수출환적(TEU)         1
수입물가지수            1
               ... 
Unnamed: 121    115
Unnamed: 122    115
Unnamed: 123    115
Unnamed: 124    115
Unnamed: 125    115
Length: 125, dtype: int64

In [42]:
# 모든 값이 전부 결측치인 열만 제거
df1 = df.dropna(axis = 1, how = 'all')

# 결측값이 있는 행 제거
df2 = df1.dropna()
# how='any' 생략
df2

,수입(TEU),수출(TEU),수입환적(TEU),수출환적(TEU),수입물가지수,usd비율(Yuan/US),Dubai국제유가(US/배럴),강수량(mm),평균기온,평균최저기온,평균최고기온
Date,,,,,,,,,,,
2014-01-01,"353,915.75","365,515.00","408,239.00","389,979.75",128.15,6.105,104.0,9.6,5.0,1.2,9.7
2014-02-01,"334,478.00","334,191.25","313,752.00","350,400.25",128.34,6.121,105.0,44.1,5.8,2.7,9.9
2014-03-01,"405,896.25","402,860.75","410,435.50","383,563.50",127.66,6.152,104.4,136.7,9.8,6.2,14.5
2014-04-01,"391,681.50","402,606.75","390,004.00","401,684.25",127.56,6.158,104.6,147.8,14.6,11.2,19.3
2014-05-01,"396,793.75","395,188.75","402,278.75","388,978.25",127.70,6.169,105.6,145.9,18.7,15.4,23.2
...,...,...,...,...,...,...,...,...,...,...,...
2023-01-01,"401,093.00","417,851.25","524,469.00","489,668.00",123.42,6.753,80.4,79.2,3.8,-0.2,8.9
2023-02-01,"410,600.50","430,718.00","442,852.00","471,788.00",123.78,6.945,82.1,30.0,6.8,2.9,11.8
2023-03-01,"493,589.25","465,645.25","550,140.00","540,680.00",121.61,6.869,78.5,76.2,12.3,8.5,17.0


# 3. 데이터 중복값 확인

In [44]:
#데이터 중복수 점검 
len(df2[df2.duplicated()])

0

# 4. 데이터 정규화 및 전처리

In [50]:
BSdata = df2

In [34]:
class bsdatamake:
    def __init__(self,BSdata, division_rate=0.7, window_step=1):
        self.m_sData = BSdata
        # BSdata : 데이터를 나타내는 변수 클레스에 저장
        self.m_division_rate = division_rate
        # division_rate: 데이터를 학습용과 테스트용으로 나누는 비율을 나타내는 변수/기본값 : 0.7 로 설정
        self.m_window_step = window_step
        #window_step: 데이터를 윈도우 스텝 크기에 맞게 분할하는 변수 / 기본값 : 1
        self.Prepare_Data()
        # 데이터 준비를 위한 메서드를 호출
       

    def Prepare_Data(self):
    # 데이터의 전처리를 수행
        self.Setup_Scaler()
        self.Divide_Data()
        #데이터 전처리를 위한 셋업과 학습,테스트 데이터 분리를 위한 메서드 호출
        
        self.m_trainX, self.m_trainY = self.Make_Dataset(self.m_TrainData)
        #학습용 데이터셋을 생성/ 입력(X)과 출력(Y) 데이터로 구성된 학습용 데이터셋을 반환
        self.m_trainX = np.reshape(self.m_trainX, (self.m_trainX.shape[0], 1, self.m_trainX.shape[10]))
        # LSTM 모델에 입력하기 위해 데이터셋의 차원 변형
        #데이터셋의 차원을 (샘플 수, 시간 단계, 특성 수)로 변형함
       
        self.m_testX, self.m_testY = self.Make_Dataset(self.m_TestData) 
        self.m_testX = np.reshape(self.m_testX, (self.m_testX.shape[0], 1, self.m_testX.shape[1]))
        #학습용과 동일하게 진행 (테스트데이터)

    #Prepare_Data 메서드를 호출하면 데이터의 전처리가 완료되고 LSTM 모델에 입력하기 위한 학습용과 테스트용 데이터셋이 생성


    
    def Setup_Scaler(self):
   #데이터 정규화 : MinMaxScaler() 사용
        self.m_Scaler = MinMaxScaler()
       
        
        self.m_sDataScaled = self.m_Scaler.fit_transform(self.m_sData)
        #fit_transform(self.m_sData) 호출: 데이터를 스케일링하고 변환한 결과를 반환
        #변환된 데이터는 self.m_sDataScaled에 저장

   #스케일링된 데이터는 이후에 학습 및 예측 과정에서 사용

  
   